## Libraries

In [709]:
import pandas as pd 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')
from neo4j import GraphDatabase

## Connect to Neo4J

In [710]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Croma2023"))
session = driver.session()

## Create entities and relate them to events

In [711]:
query = f'''MATCH (e:Event) 
where split(e.KitID, '-')[0] <> "CONT"
UNWIND e.KitID AS id_val
WITH DISTINCT id_val
MERGE (:Entity {{ID:id_val, EntityType:"Kit"}})'''
session.run(query).single()

In [712]:
query = f'''MATCH (e:Event) 
where split(e.KitID, '-')[0] = "CONT"
UNWIND e.KitID AS id_val
WITH DISTINCT id_val
MERGE (:Entity {{ID:id_val, EntityType:"Container"}})'''
session.run(query).single()

In [713]:
query = f'''MATCH (e:Event) 
UNWIND e.KitID AS kitID 
WITH e,kitID
MATCH (n:Entity) 
WHERE kitID = n.ID
MERGE (e)-[:CORR]->(n)'''
session.run(query).single()

## Create directly follows relationships between events related to the same entity

In [714]:
query = f'''MATCH (n:Entity)
MATCH (n)<-[:CORR]-(e)
WITH n, e AS nodes ORDER BY e.timestamp, ID(e)
WITH n, collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH n, event_node_list[i] AS e1, event_node_list[i+1] AS e2
MERGE (e1)-[df:DF {{EntityType:n.EntityType, ID:n.ID, Duration: duration.between(e1.timestamp , e2.timestamp).minutes}}]->(e2)
'''
session.run(query)

## Create sterilization cycles as runs for each entity

In [715]:
#add start label to Entrada Material Sucio events
query = f'''MATCH (e:Event)
WHERE e.Activity =  "Entrada Material Sucio"
OPTIONAL MATCH (f:Event) - [:DF] -> (e)
SET e:NewKitEvent, f:LastKitEvent
 '''
session.run(query).single()

In [716]:
#add end label to commisionada events
query = f'''MATCH (e:Event {{Activity:  "Comisionado"}})
OPTIONAL MATCH (f:Event) <- [:DF] - (e)
SET e:LastKitEvent, f:NewKitEvent
'''
session.run(query).single()

In [717]:
#add end label to commisionada events
query = f'''MATCH (e:Event {{Activity:  "Carga L+D liberada"}}) - [:CORR] -> (n:Entity {{EntityType:"Container"}})
OPTIONAL MATCH (f:Event) <- [:DF] - (e)
SET e:LastKitEvent, f:NewKitEvent
'''
session.run(query).single()

In [718]:
query = f'''MATCH (e:Event)
WHERE e.Activity = "Carga de esterilizador liberada"
MATCH (e) - [:DF] -> (f:Event)
where f.Activity <> "Comisionado" and 
f.Activity <> "Carga de esterilizador liberada" 
and duration.between(e.timestamp, f.timestamp).minutes > 280
SET e:LastKitEvent, f:NewKitEvent
'''
session.run(query).single()

In [719]:
#add end label to commisionada events
query = f'''MATCH (e:Event)
WHERE not () - [:DF] -> (e)
SET e:NewKitEvent
'''
session.run(query).single()

In [720]:
#add end label to commisionada events
#query = f'''MATCH (e:Event:LastKitEvent) - [:DF] -> (f:Event)
#where f.Activity <> "Comisionado"
#set f:NewKitEvent
# '''
#session.run(query).single()

In [721]:
query = f'''MATCH (f:Event) - [:DF] -> (e:Event)
where duration.between(f.timestamp, e.timestamp).days > 3
SET e:NewKitEvent, f:LastKitEvent
 '''
session.run(query).single()

In [722]:
#Find nodes that are followed by Entrada directly
query = f'''MATCH (start_event:Event:NewKitEvent:LastKitEvent)
            MATCH (start_event) - [:CORR] -> (k:Entity)
            CALL apoc.refactor.cloneNodes([k])
            YIELD input, output as new_k, error
            SET new_k:Run
            remove new_k:Entity
            SET start_event:OnlyKitEvent
            remove start_event:NewKitEvent:LastKitEvent
            //CREATE (new_k) <- [:HAS_RUN] - (k)
            WITH start_event, k, new_k
            CALL {{WITH start_event, k, new_k
            MATCH (start_event) - [r:CORR] -> (k)
            DELETE r
            CREATE (start_event) - [:CORR] -> (new_k)}}
'''
session.run(query).single()

In [723]:
#Find paths from start to end and create Run nodes
query = f'''MATCH (start_event:Event:NewKitEvent)
CALL apoc.path.expandConfig(start_event, {{relationshipFilter: "DF>", 
labelFilter:"+Event|/LastKitEvent"}})
YIELD path
MATCH (start_event) - [:CORR] -> (k:Entity)
CALL apoc.refactor.cloneNodes([k])
YIELD input, output as new_k, error
SET new_k:Run
remove new_k:Entity
//CREATE (new_k) <- [:HAS_RUN] - (k)
WITH nodes(path) as events, k, new_k
            CALL {{WITH events, k, new_k
             UNWIND events as  kit_event
            MATCH (kit_event) - [r:CORR] -> (k)
            DELETE r
            CREATE (kit_event) - [:CORR] -> (new_k)}}
 '''
session.run(query).single()

In [724]:
query = f'''match (e:Entity)
set e:Run
remove e:Entity'''
session.run(query).single()

In [725]:
query = f'''match (r:Run) 
where not (r) <- [:CORR] - ()
detach delete r
'''
session.run(query).single()

In [726]:
query = f'''match (r:Run) 
UNWIND r.ID AS id_val
WITH DISTINCT id_val, r.EntityType as et
MERGE (:Entity {{ID:id_val, EntityType : et}})
'''
session.run(query).single()

In [727]:
query = f'''MATCH (r:Run) 
UNWIND r.ID AS kitID 
WITH r, kitID
MATCH (n:Entity) 
WHERE kitID = n.ID
MERGE (r)<-[:HAS_RUN]-(n)'''
session.run(query).single()

In [728]:
query = f'''MATCH (startEvent:NewKitEvent)-[:CORR]->(c:Run) 
MATCH (endEvent:LastKitEvent)-[:CORR]->(c) 
Match path = (startEvent) - [:DF*] -> (endEvent)
set c.startTimestamp = startEvent.timestamp, 
c.endTimestamp=endEvent.timestamp, 
c.activities = startEvent.Activity
with tail(nodes(path)) as pathNodes, c
foreach (event in pathNodes | 
                set c.activities = c.activities + ", " + event.Activity)
'''
session.run(query).single()

In [729]:
query = f'''match (r:Run) <- [:CORR] - (e:Event)
where not (e) - [:DF] -> () and r.startTimestamp is null
UNWIND r.ID AS kitID
WITH r, kitID, e
MATCH (startEvent:NewKitEvent)-[:CORR]->(r) 
Match path = (startEvent) - [:DF*] -> (e)
set r.startTimestamp = startEvent.timestamp, 
r.endTimestamp=e.timestamp, 
e:LastKitEvent,
r.activities = startEvent.Activity
with tail(nodes(path)) as pathNodes, r
foreach (event in pathNodes | 
                set r.activities = r.activities + ", " + event.Activity)'''
session.run(query).single()

In [730]:
query = f'''MATCH (startEvent:OnlyKitEvent)-[:CORR]->(c:Run) 
set c.startTimestamp = startEvent.timestamp, 
c.endTimestamp=startEvent.timestamp, 
c.activities = startEvent.Activity
'''
session.run(query).single()

In [731]:
# **
query = f'''match (r:Run) <- [:CORR] - (e:Event)
where r.startTimestamp is null 
unwind r.ID as kitID
with r, count(e) as c, e
where c = 1
set e:OnlyKitEvent, 
r.startTimestamp = e.timestamp, 
r.endTimestamp = e.timestamp, 
r.activities = e.Activity
remove e:NewKitEvent:LastKitEvent'''
session.run(query).single()

In [732]:
#query = f'''
#Match (e:Entity) 
#WITH  e AS cases ORDER BY e.startTimestamp, ID(e)
#WITH collect(cases) AS cases_list
#UNWIND range(0, size(cases_list)-1) AS i
#with cases_list[i] as node, i
#Match (ev:Event) - [:CORR] -> (node)
#set node.CaseID = node.ID + "-CN" + i,
#ev.CaseID = node.ID + "-CN" + i
#'''
#session.run(query).single()

In [733]:
query = f'''
Match (e:Run) 
WITH  e AS cases ORDER BY e.startTimestamp, ID(e)
WITH collect(cases) AS cases_list
UNWIND range(0, size(cases_list)-1) AS i
with cases_list[i] as node, i
Match (ev:Event) - [:CORR] -> (node)
set node.CaseID = node.ID + "-CN" + i,
ev.CaseID = node.ID + "-CN" + i
'''
session.run(query).single()

In [734]:
query = f'''match (r:Run)
return r.CaseID, r.startTimestamp, r.endTimestamp, r.startTimestamp.month
'''
frame = pd.DataFrame(session.run(query).data())
frame['r.startTimestamp'] = pd.to_datetime(frame['r.startTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
frame['r.endTimestamp'] = pd.to_datetime(frame['r.endTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
frame['durationDays'] = (frame['r.endTimestamp'] - frame['r.startTimestamp']).dt.total_seconds() / 60 / 60 / 24
frame['durationMinutes'] = (frame['r.endTimestamp'] - frame['r.startTimestamp']).dt.total_seconds() / 60 
for index, row in frame.iterrows():
    query = f'''match (r:Run {{CaseID: "{row['r.CaseID']}"}})
                set r.durationInMinutes = {row['durationMinutes']}
            '''
    session.run(query)

In [735]:
query = f'''match (e:Entity)-[:HAS_RUN]->(r:Run)
with count(r) as numberOfCycles, e
set e.numberOfCycles = numberOfCycles
return e.ID, numberOfCycles
'''
session.run(query).single()

<Record e.ID='EQP-QUI.ORL.OPT-14' numberOfCycles=11>

## Make Kit nodes

In [736]:
query = f'''match (e:Entity {{EntityType:"Kit"}}) - [:HAS_RUN] -> (r:Run) <- [:CORR] - (ev:Event)
UNWIND ev.`Código` AS id
with distinct (id), count(distinct(ev.NS)) as num, e
merge (k:Kit {{ID :id, numberOfUnits :num}}) - [:HAS_UNIT] -> (e)
return id, num'''
session.run(query).single()

<Record id='EQP-QUI.ORL.OPT' num=1>

In [737]:
# query = f'''MATCH (e:Event)
# MATCH (n:Entity {{EntityType:"Kit"}})- [:HAS_RUN] -> (r:Run) 
# where e.KitID = n.ID
# set n.KitID = e.`Código`'''
# session.run(query).single()

In [738]:
# query = f'''MATCH (n:Entity {{EntityType:"Kit"}}) - [:HAS_RUN] -> (r:Run) 
# Match (k:Kit)
# where n.KitID = k.ID
# merge (n) <- [:HAS_UNIT] - (k)
# REMOVE n.KitID '''
# session.run(query).single()

## Make Container nodes

In [739]:
query = f'''match (e:Entity {{EntityType:"Container"}}) - [:HAS_RUN] -> (r:Run) <- [:CORR] - (ev:Event)
UNWIND ev.`Código` AS id
with distinct (id), count(distinct(ev.NS)) as num, e
merge (k:Container {{ID :id, numberOfUnits :num, isWashingMachineContainer: True}}) - [:HAS_UNIT] -> (e)
'''
session.run(query).single()

## Extract the process model

In [740]:
query = f'''MATCH ( e : Event ) WITH distinct e.Activity AS actName
MERGE ( c : Class {{ Name:actName, Type:"Activity", ID: actName}})'''
session.run(query).single()

In [741]:
query = f'''MATCH ( c : Class ) WHERE c.Type = "Activity"
MATCH ( e : Event ) WHERE c.Name = e.Activity
MERGE ( e ) -[:OBSERVED]-> ( c )'''
session.run(query).single()

In [742]:
query = f'''MATCH ( c : Class ) WHERE c.Type = "Activity"
MATCH ( e : Event ) WHERE c.Name = e.Activity
match (en:Entity) <-[:CORR]- (e)
MERGE ( en) -[:LOGGED_IN]-> ( c )'''
session.run(query).single()

In [743]:
query = f'''match (e:Entity)-[:HAS_RUN]->(r:Run)
MATCH ( c1 : Class ) <-[:OBSERVED]- ( e1 : Event ) -[df:DF]-> ( e2 : Event ) -[:OBSERVED]-> ( c2 : Class )
MATCH (e1) -[:CORR] -> (r) <-[:CORR]- (e2)
WHERE c1.Type = c2.Type AND r.EntityType = df.EntityType
WITH r.EntityType as EType,c1,count(df) AS df_freq,c2
MERGE ( c1 ) -[rel2:DF_C {{EntityType:EType}}]-> ( c2 ) ON CREATE SET rel2.count=df_freq'''
session.run(query).single()

## Add employees

In [744]:
query = f'''MATCH ( e : Event ) UNWIND e.Usuario AS employee 
WITH distinct employee
MERGE ( :Employee {{ Name:employee, Type:"Usuario", ID: employee}})'''
session.run(query).single()

In [745]:
query = f'''MATCH ( m : Employee ) WHERE m.Type = "Usuario"
MATCH ( e : Event ) WHERE m.Name = e.Usuario
CREATE ( e ) <-[:WORKED_ON]- ( m )'''
session.run(query).single()

In [746]:
query = f'''MATCH (emp:Employee)
MATCH (emp)-[:WORKED_ON]->(e)
WITH emp, e AS nodes ORDER BY e.timestamp, ID(e)
WITH emp, collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH emp, event_node_list[i] AS e1, event_node_list[i+1] AS e2
MERGE (e1)-[df:DF_EMP {{ID:emp.ID, startTimestamp: e1.timestamp, endTimestamp: e2.timestamp}}]->(e2)
'''
session.run(query).single()

## Create directly follows for activities related to a case

In [747]:
query = f'''match ( e1 : Event ) -[df:DF]-> ( e2 : Event )
WHERE e1.CaseID = e2.CaseID
MERGE ( e1 ) -[rel:DF_CASE {{CaseID: e1.CaseID}}]-> ( e2 )'''
session.run(query).single()

## Add washing machines

In [748]:
query=f'''
MATCH (e:Event) UNWIND e.additionalInfo5 AS rack
WITH DISTINCT rack, SPLIT(rack, ' - ') AS result
with rack, result, apoc.text.regexGroups(result[0], '([0-9]+)')[0][0] AS rackNumber
MERGE (:Rack {{ID:rack, name: result[0], number:rackNumber, activity: result[1], EntityType:"Rack"}})
'''
session.run(query).single()

In [749]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e.additionalInfo5 is null 
 and e.Activity = 'Cargado en carro L+D'
   and e2.Activity = 'Carga L+D iniciada'
    and e2.additionalInfo5 is not null
set e.additionalInfo5 = e2.additionalInfo5
'''
    result = session.run(query).data()
    print(result)
    if(result == []):
        flag = False
        break

[]


In [750]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e2.additionalInfo5 is null 
 and e.Activity = 'Cargado en carro L+D'
   and e2.Activity = 'Carga L+D iniciada'
    and e.additionalInfo5 is not null
set e2.additionalInfo5 = e.additionalInfo5
'''
    result = session.run(query).data()
    print(result)
    if(result == []):
        flag = False
        break

[]


In [751]:
query=f'''
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
UNWIND e.additionalInfo1 AS wm
WITH DISTINCT wm, SPLIT(wm, ' - ') AS result, e
MERGE (:WashingMachine {{activity: rtrim(result[0]), machine: COALESCE(rtrim(result[1]), rtrim(e.additionalInfo2)), EntityType:"Washing Machine"}})
'''
session.run(query).single()

In [752]:
query = f'''
Match (r:Rack) 
Match (w:WashingMachine)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with SPLIT(e.additionalInfo1, ' - ') AS wm, e.additionalInfo5 as rack, r, w
where rack = r.ID and w.activity = wm[0]
merge (r) -[:RELATED_TO] -> (w)
'''
session.run(query).single()

In [753]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e.additionalInfo1 is null 
 and e2.Activity = 'Carga L+D liberada'
   and e.Activity = 'Carga L+D iniciada'
    and e2.additionalInfo1 is not null
set e.additionalInfo1 = e2.additionalInfo1,
e.additionalInfo2 = e2.additionalInfo2,
e.additionalInfo5 = e2.additionalInfo5
'''
    result = session.run(query).data()
    print(result)
    if(result == []):
        flag = False
        break


[]


In [754]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e2.additionalInfo1 is null 
 and e2.Activity = 'Carga L+D liberada'
  and e.Activity = 'Carga L+D iniciada'
    and e.additionalInfo1 is not null
set e2.additionalInfo1 = e.additionalInfo1,
e2.additionalInfo2 = e.additionalInfo2,
e2.additionalInfo5 = e.additionalInfo5
'''
    result = session.run(query).data()
    print(result)
    if(result == []):
        flag = False
        break

[]


In [755]:
query = f'''Match (w:WashingMachine)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with SPLIT(e.additionalInfo1, ' - ') AS wm, w, e
where w.activity = wm[0]
create (e) -[:WASHED_IN {{timestamp: e.timestamp}}] -> (w)
'''
session.run(query).single()

In [756]:
query = f'''Match (r:Rack)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with e.additionalInfo5 as rack, r, e
where rack = r.ID
create (e) -[:WASHED_ON {{timestamp: e.timestamp}}] -> (r)'''
session.run(query).single()

## Add sterilization machines

In [757]:
query=f'''MATCH (e:Event) 
where e.Activity = "Composición de cargas" or e.Activity = "Carga de esterilizador liberada"
UNWIND e.additionalInfo1 AS sm
WITH DISTINCT sm, SPLIT(sm, ' - ') AS result, e
MERGE (:SterilizationMachine {{activity: rtrim(result[0]), machine: rtrim(result[1]), EntityType:"Sterilization Machine",
 type: 
 case 
 when rtrim(toLower(result[1])) in ['amsco vpro', 'amsco eagle'] then 'Low Temperature Sterilization'
     else 'High Temperature Sterilization' end}})
     '''
session.run(query).single()

In [758]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e2.additionalInfo1 is null
 and e2.Activity = 'Carga de esterilizador liberada'
   and e.Activity = 'Composición de cargas'
    and e.additionalInfo1 is not null
set e2.additionalInfo1 = e.additionalInfo1,
e2.additionalInfo2 = e.additionalInfo2
'''
    result = session.run(query).data()
    if(result == []):
        flag = False
        break


In [759]:
flag = True
while flag:
    query = f'''match (e:Event) - [:DF_CASE] -> (e2:Event)
where e.additionalInfo1 is null
 and e2.Activity = 'Carga de esterilizador liberada'
  and e.Activity = 'Composición de cargas'
   and e2.additionalInfo1 is not null
set e.additionalInfo1 = e2.additionalInfo1,
e.additionalInfo2 = e2.additionalInfo2
'''
    result = session.run(query).data()
    if(result == []):
        flag = False
        break


In [760]:
query = f'''Match (s:SterilizationMachine)
MATCH (e:Event) 
where e.Activity = "Composición de cargas" or e.Activity = "Carga de esterilizador liberada"
with SPLIT(e.additionalInfo1, ' - ') as sm, s, e
where rtrim(sm[1]) = s.machine
merge (e) -[:STERILIZED_IN {{timestamp: e.timestamp}}] -> (s)
'''
session.run(query).single()

## Create directly follows between cases

In [761]:
query = f'''match ( e1 : Event ) -[df:DF]-> ( e2 : Event )
WHERE e1.CaseID <> e2.CaseID
MERGE (e1) -[rel:DF_CYCLE {{PreviousCaseID: e1.CaseID, EndActivity: e1.Activity, endTimestamp: e1.timestamp,
 NextCaseID: e2.CaseID, StartActivity: e2.Activity, startTimestamp: e2.timestamp, 
  DurationBetweenSterilizations: Duration.between(e1.timestamp, e2.timestamp).minutes, KitID: e1.KitID}}]-> (e2)'''
session.run(query).single()

## Batch activities

In [762]:
#TODO check batches timestamp and the entry time for entrada
#TODO make separate batching relation for washing machines and sterilization machines
#TODO Question, should I batch activities with employee included or batch them as activity no matter the employee doing the job?

#query=f'''
#MATCH (e: Event)
#WITH e AS nodes ORDER BY e.timestamp, ID(e)
#WITH collect(nodes) AS event_node_list
#UNWIND range(0, size(event_node_list)-2) AS i
#WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
#WITH e1, e2, duration.between(e1.timestamp , e2.timestamp).minutes AS time_diff
#SET e1.batchID = CASE 
#    WHEN time_diff < 2 AND e1.Usuario = e2.Usuario AND e1.batchID IS NULL THEN "b" + ID(e1)
#    ELSE e1.batchID END,
#    e2.batchID = CASE 
#    WHEN time_diff < 2 AND e1.Usuario = e2.Usuario AND e1.batchID IS NOT NULL THEN e1.batchID
#    ELSE e2.batchID END
#'''
#session.run(query).single()

In [763]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
WITH e1, e2, duration.between(e1.timestamp , e2.timestamp).minutes AS time_diff
SET e1.batchID = 
CASE 
    WHEN e1.batchID IS not NULL 
        THEN e1.batchID 
    ELSE 
        "b" + ID(e1)
            END,
    e2.batchID = 
CASE 
    WHEN time_diff < 3 AND (e1) - [:DF_EMP] -> (e2) 
        THEN e1.batchID
    WHEN time_diff >= 3 AND (e1) - [:DF_EMP] -> (e2) and e2.batchID is null
        THEN "b" + ID(e2)
    else 
        e2.batchID
            END'''
session.run(query)

In [764]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
SET e.batchID = "b" + ID(e)
'''
session.run(query)

In [765]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
SET e2.batchID = 
CASE 
    WHEN duration.between(e1.timestamp , e2.timestamp).minutes < 3 AND (e1) - [:DF_EMP] -> (e2) 
        THEN e1.batchID
    else 
        e2.batchID
            END'''
session.run(query)

In [766]:
query=f'''MATCH (e: Event)
where e.Activity in ['Comisionado', 'Producción montada', 'Entrada Material Sucio', 'Montaje']
SET e.batchID = "b" + ID(e)
'''
session.run(query)

In [767]:
query=f'''MATCH (e: Event)
where e.Activity in ['Comisionado', 'Producción montada', 'Entrada Material Sucio', 'Montaje']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
SET e2.batchID = 
CASE 
    WHEN duration.between(e1.timestamp , e2.timestamp).minutes < 3 AND (e1) - [:DF_EMP] - (e2) 
        THEN e1.batchID
    else 
        e2.batchID
            END'''
session.run(query)

In [768]:
query=f'''MATCH (e:Event) 
UNWIND e.batchID AS id_val
WITH DISTINCT id_val
MERGE (:Batch {{ID:id_val}})'''
session.run(query)

In [769]:
query=f'''MATCH (e:Event) 
MATCH (b:Batch) 
WHERE e.batchID = b.ID
merge (e)-[:BATCHED]->(b)'''
session.run(query)

In [770]:
query = f'''MATCH (e:Event)-[:BATCHED]->(b:Batch)
WITH e.timestamp AS timestamp, b
with MIN(timestamp) AS minTimestamp, MAX(timestamp) AS maxTimestamp, b
set b.startTimestamp = minTimestamp, b.endTimestamp = maxTimestamp'''
session.run(query)

In [771]:
#query = f'''match (s:SterilizationMachine) <- [si:STERILIZED_IN] - (e:Event) - [:DF_CASE] -> (e2:Event)
#where e2.additionalInfo1 is null and e2.Activity = 'Carga de esterilizador liberada' and  e.Activity = 'Composición de cargas'
#set e2.additionalInfo1 = e.additionalInfo1,
#e2.additionalInfo2 = e.additionalInfo2
#create (e2) -[:STERILIZED_IN {{timestamp: e2.timestamp}}] -> (s)'''
#session.run(query)

In [772]:
#
# query=f'''MATCH (e: Event)
# where e.Activity in ['Composición de cargas', 'Carga de esterilizador liberada']
# WITH e AS nodes ORDER BY e.timestamp, ID(e)
# WITH collect(nodes) AS event_node_list
# UNWIND range(0, size(event_node_list)-2) AS i
# WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
# where e1.batchID = e2.batchID and e1.additionalInfo1 is null and e2.additionalInfo1 is not null
# set e1.additionalInfo1 = e2.additionalInfo1'''
# session.run(query).single()

## Create Contained Kits

In [773]:
query = f'''match (e:NewKitEvent {{Activity: "Montaje"}})
WITH collect(distinct e.KitID) as kids 
UNWIND range(0, size(kids)-1) AS i
with kids[i] as kid
match (k:Kit) - [:HAS_UNIT] -> (e:Entity {{ID: kid}})
set k.isContainedKit = True'''
session.run(query).single()

In [774]:
query = f'''match (k:Kit {{isContainedKit: True}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
where (r.activities contains 'Entrada Material Sucio' 
or r.activities contains 'Cargado en carro L+D' 
or r.activities contains 'Carga L+D iniciada' 
or r.activities contains 'Carga L+D liberada')
set k.isContainedKit = False'''
session.run(query).single()

In [775]:
query = f'''match (k:Kit {{isContainedKit: True}}) - [:HAS_UNIT] -> (e:Entity) <- [:CORR] - (ev:Event)
where (ev.Activity = 'Entrada Material Sucio' 
or ev.Activity = 'Cargado en carro L+D' 
or ev.Activity = 'Carga L+D iniciada'
or ev.Activity = 'Carga L+D liberada')
set k.isContainedKit = False'''
session.run(query).single()

## Relate Kits to Containers

In [776]:
flag = True
while flag:
    query = f'''match (c:Container)
with c.ID as text
with split(text,'-')[1] as part2
with collect(distinct part2) as kcol
match (k:Kit {{isContainedKit: true}})
WITH k.ID AS text, kcol
WITH SPLIT(text, '-')[0] AS part1,
       SPLIT(SPLIT(text, '-')[1], '.')[0] + '.' + SPLIT(SPLIT(text, '-')[1], '.')[1] AS part2,
       SPLIT(SPLIT(text, '-')[1], '.')[2] AS part3, kcol
where part2 in kcol
unwind part2 as kid
with distinct kid
match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
WHERE k.ID contains kid and r.durationInMinutes > 0 and r.containerCaseID is null
match (c:Container) - [:HAS_UNIT] -> (e2:Entity) - [:HAS_RUN] -> (r2:Run)
where c.ID contains kid and r2.durationInMinutes > 0 AND r.startTimestamp >= r2.endTimestamp
with r, r2, k, c
ORDER BY r2.endTimestamp desc
LIMIT 1
set r.containerCaseID = r2.CaseID, 
k.isLinked = true,
c.isLinked = true
MERGE (r2)-[:CONTAIN_RUN]->(r)
return r.CaseID'''
    result = session.run(query).data()
    if(result == []):
        flag = False
        break

In [777]:
flag = True
while flag:
    query = f'''
match (c:Container)
where c.ID =~ '^[^.]*$'
with c.ID as text
with split(text, '-')[1] as part2
with collect(distinct part2) as kcol
match (k:Kit {{isContainedKit: true}})
where k.ID =~ '^[^.]*\.[^.]*$'
WITH k.ID AS text, kcol
WITH split(split(text, '-')[1], '.')[0] AS part2, kcol
where part2 in kcol
unwind part2 as kid
with distinct kid
match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
WHERE split(split(k.ID, '-')[1], '.')[0] = kid 
and r.durationInMinutes > 0 
and r.containerCaseID is null 
and split(r.activities, ', ')[0] = "Montaje"
match (c:Container) - [:HAS_UNIT] -> (e2:Entity) - [:HAS_RUN] -> (r2:Run)
where split(c.ID, '-')[1] contains kid 
and r2.durationInMinutes > 0 
AND r.startTimestamp >= r2.endTimestamp
and split(r2.activities, ', ')[size(split(r2.activities, ', ')) - 1] = "Carga L+D liberada" 
with r, r2, k, c
ORDER BY r2.endTimestamp desc
LIMIT 1
set r.containerCaseID = r2.CaseID,
k.isLinked = true,
c.isLinked = true
MERGE (r2)-[:CONTAIN_RUN]->(r)
return r.CaseID
'''
    result = session.run(query).data()
    if(result == []):
        flag = False
        break

In [778]:
# flag = True
# while flag:
#     query = f'''
# match (k:Kit {{isContainedKit: true}})
# where k.ID =~ '^[^.]*\.[^.]*$'
# WITH k.ID AS text
# WITH split(split(text, '-')[1], '.')[0] AS part2
# unwind part2 as kid
# with distinct kid
# match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
# WHERE split(split(k.ID, '-')[1], '.')[0] = kid 
# and r.durationInMinutes > 0 
# and r.containerCaseID is null 
# and split(r.activities, ', ')[0] = "Montaje"
# match (c:Container) - [:HAS_UNIT] -> (e2:Entity) - [:HAS_RUN] -> (r2:Run)
# where c.ID =~ '^[^.]*$' and split(c.ID, '-')[1] contains kid 
# and r2.durationInMinutes > 0 
# AND r.startTimestamp >= r2.endTimestamp
# and split(r2.activities, ', ')[size(split(r2.activities, ', ')) - 1] = "Carga L+D liberada" 
# with r, r2, k, c
# ORDER BY r2.endTimestamp desc
# LIMIT 1
# set r.containerCaseID = r2.CaseID,
# k.isLinked = true, 
# c.isLinked = true
# MERGE (r2)-[:CONTAIN_RUN]->(r)
# return r.CaseID
# '''
#     result = session.run(query).data()
#     print(result)
#     if(result == []):
#         flag = False
#         break

In [779]:
# query = f'''MATCH p=(c:Container {{isTest:True}}) - [:HAS_UNIT] -> (:Entity) - [:HAS_RUN] -> (r2:Run)-[cr:CONTAIN_RUN] ->(r:Run) <- [:HAS_RUN] - (:Entity) <- [:HAS_UNIT] -(k:Kit {{isTest:True}}) 
# remove c.isTest, k.isTest, r.isTest, r2.isTest , r.containerCaseID, k.isLinked, c.isLinked
# delete cr'''
# session.run(query)

In [780]:
# flag = True
# while flag:
#     query = f'''
# match (k:Kit {{isContainedKit: true}})
# where k.ID =~ '^[^.]*\.[^.]*\.[^.]*$'
# WITH k.ID AS text
# WITH split(split(text, '-')[1], '.')[1] AS part2
# unwind part2 as kid
# with distinct kid
# match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
# WHERE split(split(k.ID, '-')[1], '.')[1] = kid 
# and r.durationInMinutes > 0 
# and r.containerCaseID is null 
# and split(r.activities, ', ')[0] = "Montaje"
# match (c:Container) - [:HAS_UNIT] -> (e2:Entity) - [:HAS_RUN] -> (r2:Run)
# where c.ID =~ '^[^.]*$' and split(c.ID, '-')[1] contains kid 
# and r2.durationInMinutes > 0 
# AND r.startTimestamp >= r2.endTimestamp
# and split(r2.activities, ', ')[size(split(r2.activities, ', ')) - 1] = "Carga L+D liberada" 
# with r, r2, k, c
# ORDER BY r2.endTimestamp desc
# LIMIT 1
# set r.containerCaseID = r2.CaseID, r.isTest = true,
# k.isLinked = true, k.isTest = true,
# c.isLinked = true, c.isTest = true
# MERGE (r2)-[:CONTAIN_RUN {{isTest:true}}]->(r)
# return r.CaseID
# '''
#     result = session.run(query).data()
#     print(result)
#     if(result == []):
#         flag = False
#         break

In [781]:
def group_consecutive_ones(row):
    start = None
    prevIsKit = True
    cont_records1 = pd.DataFrame(columns=['CaseID'])
    non_cont_records1 = pd.DataFrame(columns=['CaseID'])
    for i, value in enumerate(row):
        if value.startswith("CONT") and prevIsKit and start is None:
            start = i
            prevIsKit = False
            cont_records1 = cont_records1.append({'CaseID': value}, ignore_index=True)
        elif value.startswith("CONT") and not prevIsKit and start is not None:
            cont_records1 = cont_records1.append({'CaseID': value}, ignore_index=True)
        elif value.startswith("CONT") and prevIsKit and start is not None:
            while not non_cont_records1.empty:
                selected_row = non_cont_records1.sample(n=1)
                selected_cont_records1 = cont_records1.sample(n=1, replace=True)
                non_cont_records1 = non_cont_records1.drop(selected_row.index)
                query = f'''match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run {{CaseID: "{selected_row['CaseID'].values[0]}"}})
                            match (c:Container) - [:HAS_UNIT] -> (e2:Entity) - [:HAS_RUN] -> (r2:Run {{CaseID: "{selected_cont_records1['CaseID'].values[0]}"}})
                            with r, r2, k, c
                            set r.containerCaseID = r2.CaseID,
                            k.isLinked = true, 
                            c.isLinked = true
                            MERGE (r2)-[:CONTAIN_RUN]->(r)'''
                session.run(query)
            start = None
            prevIsKit = True
            cont_records1 = pd.DataFrame(columns=['CaseID'])
            cont_records1 = cont_records1.append({'CaseID': value}, ignore_index=True)
        elif not value.startswith("CONT"):
            prevIsKit = True
            non_cont_records1 = non_cont_records1.append({'CaseID': value}, ignore_index=True)


In [782]:
query = f'''
match (k:Kit {{isContainedKit: true}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
WHERE r.durationInMinutes > 0 and r.containerCaseID is null and split(r.activities, ', ')[0] = "Montaje"
return r.CaseID, r.activities, r.startTimestamp, r.endTimestamp
union
match (c:Container) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
where c.isLinked is null and r.durationInMinutes > 0 and split(r.activities, ', ')[size(split(r.activities, ', ')) - 1] = "Carga L+D liberada"
return r.CaseID, r.activities, r.startTimestamp, r.endTimestamp
ORDER BY r.startTimestamp
'''
result = session.run(query)
records = [record.values() for record in result]
columns = result.keys()
df = pd.DataFrame(records, columns=columns)
df['r.startTimestamp'] = pd.to_datetime(df['r.startTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
df['r.endTimestamp'] = pd.to_datetime(df['r.endTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
df = df.sort_values('r.startTimestamp')
group_consecutive_ones(df['r.CaseID'])

In [783]:
query = f'''MATCH p=(e:LastKitEvent) - [:CORR] -> (r1:Run) - [r:CONTAIN_RUN]-> (r2:Run) <- [:CORR] - (e2:NewKitEvent) 
            create (e) - [:DF_CASE {{inferred:true}}] -> (e2)
            set e:ConnectionKitEvent, e2:ConnectionKitEvent'''
result = session.run(query)


In [784]:

query = f'''match (r:Run)
            where r.containerCaseID is not null
            unwind r.containerCaseID as cid
            with distinct cid
            match (e:Event {{CaseID: cid}})
            match (r:Run {{containerCaseID: cid}})
            merge (e) - [:CORR {{inferred:true}}] -> (r)'''
result = session.run(query)


In [785]:

query = f'''MATCH (startEvent:NewKitEvent)-[:CORR]->(c:Run) 
            where not startEvent:ConnectionKitEvent and c.containerCaseID is not null
            MATCH (endEvent:LastKitEvent)-[:CORR]->(c) 
            where not endEvent:ConnectionKitEvent
            Match path = (startEvent) - [:DF_CASE*] -> (endEvent)
            set c.startTimestamp = startEvent.timestamp, 
            c.endTimestamp=endEvent.timestamp, 
            c.activities = startEvent.Activity
            with tail(nodes(path)) as pathNodes, c
            foreach (event in pathNodes | 
                set c.activities = c.activities + ", " + event.Activity)'''
result = session.run(query)


## Construct Waiting time and Working Time 

In [789]:
query = f'''match (e1:Event {{Activity:"Entrada Material Sucio"}}) - [df:DF_CASE] -> 
(e2:Event {{Activity:"Cargado en carro L+D"}}) 
with duration.between(e1.timestamp, e2.timestamp).minutes as waitingDurationInMinutes, df
set df.waitingDurationInMinutes = waitingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Cargado en carro L+D"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Carga L+D iniciada"}})
with duration.between(e1.timestamp, e2.timestamp).minutes as waitingDurationInMinutes, df
set df.waitingDurationInMinutes = waitingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Carga L+D iniciada"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Carga L+D liberada"}}) 
with duration.between(e1.timestamp, e2.timestamp).minutes as workingDurationInMinutes, df
set df.workingDurationInMinutes = workingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Carga L+D liberada"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Montaje"}}) 
with duration.between(e1.timestamp, e2.timestamp).minutes as waitingDurationInMinutes, df
set df.waitingDurationInMinutes = waitingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Montaje"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Producción montada"}})
with duration.between(e1.timestamp, e2.timestamp).minutes as workingDurationInMinutes, df
set df.workingDurationInMinutes = workingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Producción montada"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Composición de cargas"}})
with duration.between(e1.timestamp, e2.timestamp).minutes as waitingDurationInMinutes, df
set df.waitingDurationInMinutes = waitingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Composición de cargas"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Carga de esterilizador liberada"}})
with duration.between(e1.timestamp, e2.timestamp).minutes as workingDurationInMinutes, df
set df.workingDurationInMinutes = workingDurationInMinutes
'''
session.run(query)
query = f'''match (e1:Event {{Activity:"Carga de esterilizador liberada"}}) - [df:DF_CASE] -> (e2:Event {{Activity:"Comisionado"}})
with duration.between(e1.timestamp, e2.timestamp).minutes as waitingDurationInMinutes, df
set df.waitingDurationInMinutes = waitingDurationInMinutes
'''
session.run(query)


In [791]:
query = f'''
MATCH (e1:Event)-[df:DF_CASE]->(e2:Event)
match (e1) - [:CORR] -> (r:Run) <- [:CORR] - (e2)
WITH r, SUM(df.workingDurationInMinutes) AS totalValue
SET r.workingDurationInMinutes = totalValue
'''
session.run(query)

query = f'''
MATCH (e1:Event)-[df:DF_CASE]->(e2:Event)
match (e1) - [:CORR] -> (r:Run) <- [:CORR] - (e2)
WITH r, SUM(df.waitingDurationInMinutes) AS totalValue
SET r.waitingDurationInMinutes = totalValue
'''
session.run(query)